# history dumps import to mariadb

## imports

### libraries, datapath, metadata

In [63]:
import os, re, pandas as pd, numpy as np
from datetime import datetime as dt

In [4]:
datapath = '../data/raw/jawiki/dumps_unzipped/'

In [5]:
mtdt = pd.read_csv('../references/metadata/history.tsv', sep='\t')
# print(mtdt.col_name.tolist())

## make queries for raw import from dumps to mysql

### code to make queries

#### drop table

```sql
DROP TABLE IF EXISTS ja_hist;
```

#### create table

```sql
CREATE TABLE ja_hist (  wiki_db VARCHAR(20) NOT NULL, event_entity VARCHAR(20) NOT NULL, event_type VARCHAR(20) NOT NULL, event_timestamp VARCHAR(30) NOT NULL, event_comment TEXT, event_user_id BIGINT, event_user_text_historical TEXT, event_user_text TEXT, event_user_blocks_historical VARCHAR(30), event_user_blocks VARCHAR(60), event_user_groups_historical TEXT, event_user_groups TEXT, event_user_is_bot_by_historical VARCHAR(10), event_user_is_bot_by VARCHAR(10), event_user_is_created_by_self VARCHAR(10),  event_user_is_created_by_system VARCHAR(10),  event_user_is_created_by_peer VARCHAR(10),  event_user_is_anonymous VARCHAR(10),  event_user_registration_timestamp VARCHAR(30), event_user_creation_timestamp VARCHAR(30), event_user_first_edit_timestamp VARCHAR(30), event_user_revision_count BIGINT, event_user_seconds_since_previous_revision BIGINT, page_id BIGINT, page_title_historical VARCHAR(200), page_title VARCHAR(200), page_namespace_historical INT, page_namespace_is_content_historical VARCHAR(10),  page_namespace INT, page_namespace_is_content VARCHAR(10),  page_is_redirect VARCHAR(20),  page_is_deleted VARCHAR(20),  page_creation_timestamp VARCHAR(30), page_first_edit_timestamp VARCHAR(30), page_revision_count BIGINT, page_seconds_since_previous_revision BIGINT, user_id BIGINT, user_text_historical VARCHAR(100), user_text VARCHAR(100), user_blocks_historical VARCHAR(50), user_blocks VARCHAR(50), user_groups_historical VARCHAR(50), user_groups VARCHAR(50), user_is_bot_by_historical VARCHAR(50), user_is_bot_by VARCHAR(50), user_is_created_by_self VARCHAR(50),  user_is_created_by_system VARCHAR(50),  user_is_created_by_peer VARCHAR(50),  user_is_anonymous VARCHAR(10),  user_registration_timestamp VARCHAR(30), user_creation_timestamp VARCHAR(30), user_first_edit_timestamp VARCHAR(30), revision_id BIGINT, revision_parent_id BIGINT, revision_minor_edit VARCHAR(10),  revision_deleted_parts VARCHAR(30), revision_deleted_parts_are_suppressed VARCHAR(10),  revision_text_bytes BIGINT, revision_text_bytes_diff BIGINT, revision_text_sha1 VARCHAR(50), revision_content_model VARCHAR(10),  revision_content_format VARCHAR(10),  revision_is_deleted_by_page_deletion VARCHAR(10),  revision_deleted_by_page_deletion_timestamp VARCHAR(30), revision_is_identity_reverted VARCHAR(10),  revision_first_identity_reverting_revision_id BIGINT, revision_seconds_to_identity_revert BIGINT, revision_is_identity_revert VARCHAR(10),  revision_is_from_before_page_creation VARCHAR(10),  revision_tags VARCHAR(50), row_id BIGINT NOT NULL AUTO_INCREMENT, PRIMARY KEY (row_id) );
```

In [58]:
create_table = '''CREATE TABLE ja_hist (  wiki_db VARCHAR(20) NOT NULL, event_entity VARCHAR(20) NOT NULL, event_type VARCHAR(20) NOT NULL, event_timestamp VARCHAR(30) NOT NULL, event_comment TEXT, event_user_id BIGINT, event_user_text_historical TEXT, event_user_text TEXT, event_user_blocks_historical VARCHAR(30), event_user_blocks VARCHAR(60), event_user_groups_historical TEXT, event_user_groups TEXT, event_user_is_bot_by_historical VARCHAR(10), event_user_is_bot_by VARCHAR(10), event_user_is_created_by_self VARCHAR(10),  event_user_is_created_by_system VARCHAR(10),  event_user_is_created_by_peer VARCHAR(10),  event_user_is_anonymous VARCHAR(10),  event_user_registration_timestamp VARCHAR(30), event_user_creation_timestamp VARCHAR(30), event_user_first_edit_timestamp VARCHAR(30), event_user_revision_count BIGINT, event_user_seconds_since_previous_revision BIGINT, page_id BIGINT, page_title_historical VARCHAR(200), page_title VARCHAR(200), page_namespace_historical INT, page_namespace_is_content_historical VARCHAR(10),  page_namespace INT, page_namespace_is_content VARCHAR(10),  page_is_redirect VARCHAR(20),  page_is_deleted VARCHAR(20),  page_creation_timestamp VARCHAR(30), page_first_edit_timestamp VARCHAR(30), page_revision_count BIGINT, page_seconds_since_previous_revision BIGINT, user_id BIGINT, user_text_historical VARCHAR(100), user_text VARCHAR(100), user_blocks_historical VARCHAR(50), user_blocks VARCHAR(50), user_groups_historical VARCHAR(50), user_groups VARCHAR(50), user_is_bot_by_historical VARCHAR(50), user_is_bot_by VARCHAR(50), user_is_created_by_self VARCHAR(50),  user_is_created_by_system VARCHAR(50),  user_is_created_by_peer VARCHAR(50),  user_is_anonymous VARCHAR(10),  user_registration_timestamp VARCHAR(30), user_creation_timestamp VARCHAR(30), user_first_edit_timestamp VARCHAR(30), revision_id BIGINT, revision_parent_id BIGINT, revision_minor_edit VARCHAR(10),  revision_deleted_parts VARCHAR(30), revision_deleted_parts_are_suppressed VARCHAR(10),  revision_text_bytes BIGINT, revision_text_bytes_diff BIGINT, revision_text_sha1 VARCHAR(50), revision_content_model VARCHAR(10),  revision_content_format VARCHAR(10),  revision_is_deleted_by_page_deletion VARCHAR(10),  revision_deleted_by_page_deletion_timestamp VARCHAR(30), revision_is_identity_reverted VARCHAR(10),  revision_first_identity_reverting_revision_id BIGINT, revision_seconds_to_identity_revert BIGINT, revision_is_identity_revert VARCHAR(10),  revision_is_from_before_page_creation VARCHAR(10),  revision_tags VARCHAR(50), row_id BIGINT NOT NULL AUTO_INCREMENT, PRIMARY KEY (row_id) );'''

#### load data local infile

##### make the query

In [66]:
colnames = ['wiki_db', 'event_entity', 'event_type', 'event_timestamp', 'event_comment', 'event_user_id', 'event_user_text_historical', 'event_user_text', 'event_user_blocks_historical', 'event_user_blocks', 'event_user_groups_historical', 'event_user_groups', 'event_user_is_bot_by_historical', 'event_user_is_bot_by', 'event_user_is_created_by_self', 'event_user_is_created_by_system', 'event_user_is_created_by_peer', 'event_user_is_anonymous', 'event_user_registration_timestamp', 'event_user_creation_timestamp', 'event_user_first_edit_timestamp', 'event_user_revision_count', 'event_user_seconds_since_previous_revision', 'page_id', 'page_title_historical', 'page_title', 'page_namespace_historical', 'page_namespace_is_content_historical', 'page_namespace', 'page_namespace_is_content', 'page_is_redirect', 'page_is_deleted', 'page_creation_timestamp', 'page_first_edit_timestamp', 'page_revision_count', 'page_seconds_since_previous_revision', 'user_id', 'user_text_historical', 'user_text', 'user_blocks_historical', 'user_blocks', 'user_groups_historical', 'user_groups', 'user_is_bot_by_historical', 'user_is_bot_by', 'user_is_created_by_self', 'user_is_created_by_system', 'user_is_created_by_peer', 'user_is_anonymous', 'user_registration_timestamp', 'user_creation_timestamp', 'user_first_edit_timestamp', 'revision_id', 'revision_parent_id', 'revision_minor_edit', 'revision_deleted_parts', 'revision_deleted_parts_are_suppressed', 'revision_text_bytes', 'revision_text_bytes_diff', 'revision_text_sha1', 'revision_content_model', 'revision_content_format', 'revision_is_deleted_by_page_deletion', 'revision_deleted_by_page_deletion_timestamp', 'revision_is_identity_reverted', 'revision_first_identity_reverting_revision_id', 'revision_seconds_to_identity_revert', 'revision_is_identity_revert', 'revision_is_from_before_page_creation', 'revision_tags', 'row_id']

In [89]:
years = list(range(2013,2022))
tsvpaths = [f'/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.{year}.tsv' for year in years]

In [91]:
load_data_query_list = ([
r'''
LOAD DATA LOCAL INFILE ''' + f"'{tsvpath}' " + 
'''
    INTO TABLE ja_hist  
    CHARACTER SET utf8mb4 
    FIELDS TERMINATED BY '\t' 
    LINES TERMINATED BY '\n'

''' + '(@' + ',@'.join(colnames) + ')' +  # columns as @variables
'''

SET
'''
+ ','.join([f"{i} = NULLIF(@{i},'')" for i in colnames]) + ';'  # recieve empty strings to NULL values
for tsvpath in tsvpaths
])

##### previously-working text of load data local infile

```sql
LOAD DATA LOCAL INFILE '/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.2014.tsv' 
    INTO TABLE ja_hist  
    CHARACTER SET utf8mb4 
    FIELDS TERMINATED BY '\t' 
    LINES TERMINATED BY '\n' 
    (@wiki_db,@event_entity,@event_type,@event_timestamp,@event_comment,@event_user_id,@event_user_text_historical,@event_user_text,@event_user_blocks_historical,@event_user_blocks,@event_user_groups_historical,@event_user_groups,@event_user_is_bot_by_historical,@event_user_is_bot_by,@event_user_is_created_by_self,@event_user_is_created_by_system,@event_user_is_created_by_peer,@event_user_is_anonymous,@event_user_registration_timestamp,@event_user_creation_timestamp,@event_user_first_edit_timestamp,@event_user_revision_count,@event_user_seconds_since_previous_revision,@page_id,@page_title_historical,@page_title,@page_namespace_historical,@page_namespace_is_content_historical,@page_namespace,@page_namespace_is_content,@page_is_redirect,@page_is_deleted,@page_creation_timestamp,@page_first_edit_timestamp,@page_revision_count,@page_seconds_since_previous_revision,@user_id,@user_text_historical,@user_text,@user_blocks_historical,@user_blocks,@user_groups_historical,@user_groups,@user_is_bot_by_historical,@user_is_bot_by,@user_is_created_by_self,@user_is_created_by_system,@user_is_created_by_peer,@user_is_anonymous,@user_registration_timestamp,@user_creation_timestamp,@user_first_edit_timestamp,@revision_id,@revision_parent_id,@revision_minor_edit,@revision_deleted_parts,@revision_deleted_parts_are_suppressed,@revision_text_bytes,@revision_text_bytes_diff,@revision_text_sha1,@revision_content_model,@revision_content_format,@revision_is_deleted_by_page_deletion,@revision_deleted_by_page_deletion_timestamp,@revision_is_identity_reverted,@revision_first_identity_reverting_revision_id,@revision_seconds_to_identity_revert,@revision_is_identity_revert,@revision_is_from_before_page_creation,@revision_tags,@row_id) 

SET wiki_db = NULLIF(@wiki_db,''), event_entity = NULLIF(@event_entity,''), event_type = NULLIF(@event_type,''), event_timestamp = NULLIF(@event_timestamp,''), event_comment = NULLIF(@event_comment,''), event_user_id = NULLIF(@event_user_id,''), event_user_text_historical = NULLIF(@event_user_text_historical,''), event_user_text = NULLIF(@event_user_text,''), event_user_blocks_historical = NULLIF(@event_user_blocks_historical,''), event_user_blocks = NULLIF(@event_user_blocks,''), event_user_groups_historical = NULLIF(@event_user_groups_historical,''), event_user_groups = NULLIF(@event_user_groups,''), event_user_is_bot_by_historical = NULLIF(@event_user_is_bot_by_historical,''), event_user_is_bot_by = NULLIF(@event_user_is_bot_by,''), event_user_is_created_by_self = NULLIF(@event_user_is_created_by_self,''), event_user_is_created_by_system = NULLIF(@event_user_is_created_by_system,''), event_user_is_created_by_peer = NULLIF(@event_user_is_created_by_peer,''), event_user_is_anonymous = NULLIF(@event_user_is_anonymous,''), event_user_registration_timestamp = NULLIF(@event_user_registration_timestamp,''), event_user_creation_timestamp = NULLIF(@event_user_creation_timestamp,''), event_user_first_edit_timestamp = NULLIF(@event_user_first_edit_timestamp,''), event_user_revision_count = NULLIF(@event_user_revision_count,''), event_user_seconds_since_previous_revision = NULLIF(@event_user_seconds_since_previous_revision,''), page_id = NULLIF(@page_id,''), page_title_historical = NULLIF(@page_title_historical,''), page_title = NULLIF(@page_title,''), page_namespace_historical = NULLIF(@page_namespace_historical,''), page_namespace_is_content_historical = NULLIF(@page_namespace_is_content_historical,''), page_namespace = NULLIF(@page_namespace,''), page_namespace_is_content = NULLIF(@page_namespace_is_content,''), page_is_redirect = NULLIF(@page_is_redirect,''), page_is_deleted = NULLIF(@page_is_deleted,''), page_creation_timestamp = NULLIF(@page_creation_timestamp,''), page_first_edit_timestamp = NULLIF(@page_first_edit_timestamp,''), page_revision_count = NULLIF(@page_revision_count,''), page_seconds_since_previous_revision = NULLIF(@page_seconds_since_previous_revision,''), user_id = NULLIF(@user_id,''), user_text_historical = NULLIF(@user_text_historical,''), user_text = NULLIF(@user_text,''), user_blocks_historical = NULLIF(@user_blocks_historical,''), user_blocks = NULLIF(@user_blocks,''), user_groups_historical = NULLIF(@user_groups_historical,''), user_groups = NULLIF(@user_groups,''), user_is_bot_by_historical = NULLIF(@user_is_bot_by_historical,''), user_is_bot_by = NULLIF(@user_is_bot_by,''), user_is_created_by_self = NULLIF(@user_is_created_by_self,''), user_is_created_by_system = NULLIF(@user_is_created_by_system,''), user_is_created_by_peer = NULLIF(@user_is_created_by_peer,''), user_is_anonymous = NULLIF(@user_is_anonymous,''), user_registration_timestamp = NULLIF(@user_registration_timestamp,''), user_creation_timestamp = NULLIF(@user_creation_timestamp,''), user_first_edit_timestamp = NULLIF(@user_first_edit_timestamp,''), revision_id = NULLIF(@revision_id,''), revision_parent_id = NULLIF(@revision_parent_id,''), revision_minor_edit = NULLIF(@revision_minor_edit,''), revision_deleted_parts = NULLIF(@revision_deleted_parts,''), revision_deleted_parts_are_suppressed = NULLIF(@revision_deleted_parts_are_suppressed,''), revision_text_bytes = NULLIF(@revision_text_bytes,''), revision_text_bytes_diff = NULLIF(@revision_text_bytes_diff,''), revision_text_sha1 = NULLIF(@revision_text_sha1,''), revision_content_model = NULLIF(@revision_content_model,''), revision_content_format = NULLIF(@revision_content_format,''), revision_is_deleted_by_page_deletion = NULLIF(@revision_is_deleted_by_page_deletion,''), revision_deleted_by_page_deletion_timestamp = NULLIF(@revision_deleted_by_page_deletion_timestamp,''), revision_is_identity_reverted = NULLIF(@revision_is_identity_reverted,''), revision_first_identity_reverting_revision_id = NULLIF(@revision_first_identity_reverting_revision_id,''), revision_seconds_to_identity_revert = NULLIF(@revision_seconds_to_identity_revert,''), revision_is_identity_revert = NULLIF(@revision_is_identity_revert,''), revision_is_from_before_page_creation = NULLIF(@revision_is_from_before_page_creation,''), revision_tags = NULLIF(@revision_tags,''), row_id = NULLIF(@row_id,'');
```

### load mysql connector

### run load data

## preliminary eda in sql

##### ```revision_minor_edit```

```sql
select revision_minor_edit, min(row_id), count(*) 
    from ja_hist 
    group by revision_minor_edit order by row_id;
```

##### ```user_is_anonymous```

```sql
select user_is_anonymous, min(row_id), count(*) 
    from ja_hist 
    group by user_is_anonymous order by row_id;
```

##### ```user_text_historical```

```sql
select user_text_historical, min(row_id), count(*) 
    from ja_hist 
    where user_text_historical regexp '^[0-9]*$' 
    group by user_text_historical order by row_id;
```

##### ```revision_content_model```

##### ```user_is_bot_by_historical```

##### ```page_revision_count```

## <mark>need to refactor/reorganize</mark>

### head

### split file so I can run csvsql to generate schema (now unused)

#### confirm the rows were copied

#### hand-edited query

#### simpler table

### create table ja_hist

#### imports, login

In [36]:
import mysql.connector as mysql
import sqlalchemy

```bash
mysql --user=root --password=XXXXXXXX
```

```sql
CREATE DATABASE ja_hist CHARACTER SET utf8 COLLATE utf8_bin;
USE ja_wiki;
GRANT ALL PRIVILEGES ON ja_hist TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
GRANT ALL PRIVILEGES ON ja_hist.* TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
EXIT
```

In [14]:
mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')

#### connect to database

In [37]:
host='localhost'; user=mysql_superuser; passwd=mysql_su_pass; dbname='ja_hist';
cxn = mysql.connect(host=host,user=user,passwd=passwd, database=dbname)
cur = cxn.cursor()

connection_str = 'mysql+mysqlconnector://'+user+':'+passwd+'@'+host+'/'+dbname  # removed this after host +':'+dbport
try:
    engine1 = sqlalchemy.create_engine(connection_str)
    conn1 = engine1.connect()
except:
    print('Database connection error - check creds')
metadata = sqlalchemy.MetaData(conn1)
metadata.reflect()
metadata.tables.keys()

dict_keys(['ja_hist'])

#### create table with schema

In [38]:
conn1.execute('DROP TABLE IF EXISTS ja_hist;')
conn1.execute(make_history_table)

In [41]:
a = conn1.execute('describe ja_hist;')
a.all();

### insert data

Reference: ([link](https://mariadb.com/kb/en/load-data-infile/))  
```sql
LOAD DATA INFILE 'ld.txt' INTO TABLE ld (@i,v) SET i=@i*2;
```

In [44]:
load_data_sql = """
LOAD DATA LOCAL INFILE '/home/bhrdwj/git/predwikt/data/raw/jawiki/dumps_unzipped/2021-12.jawiki.2013.tsv'
INTO TABLE ja_hist 
CHARACTER SET utf8mb4
FIELDS TERMINATED BY "\t";
"""

In [45]:
conn1.execute(load_data_sql)

START 19:24PM

In [46]:
tblname='ja_hist'
sql_eda2 = f'select count(*) from {tblname};'
print(dt.now())
sql_rows = pd.read_sql(sql_eda2,engine1)
print(sql_rows)
print(dt.now())

2022-01-09 16:01:32.096460
   count(*)
0         0
2022-01-09 16:01:35.371606


In [30]:
a = conn1.execute('select * from ja_hist;')
a.fetchall()

[]

In [ ]:
conn1.execute('insert ')

# end